# Looped image_extractor no ROI

In [27]:
outpath = "/Users/issac/Documents/Projects/nano_strong_hackathon/slide_viewer/out_high_res/"#"/Volumes/Elements/nanostring_hackaton/H_and_E/output_wc/"
relevant_path = "/Volumes/Elements/nanostring_hackaton/high_res_scans/"

In [30]:
from pathlib import Path
from plotly.offline import plot
import pandas as pd
import plotly.graph_objs as go
import os
import openslide
import re


outpath =  Path(outpath)
if not outpath.exists():
    os.mkdir(outpath)
included_extensions = [".tif"]
excluded_str = ["._"]
file_names = [fn for fn in os.listdir(relevant_path) if any(fn.endswith(ext) for ext in included_extensions)]
file_names = [fn for fn in file_names if any(not fn.startswith(ext) for ext in excluded_str)]

for i in file_names:
    scan_name = i.replace(".tif","")
    tif_locale = relevant_path + "/" + i
    plot_size = 1000
    scale_factor = 1

    plot_size = plot_size
    slide = openslide.open_slide(tif_locale)
    base_dims = slide.level_dimensions[-1] #-2 for 2nd smallest
    base_ds = slide.level_downsamples[-1]
    img_arr = slide.read_region((0,0), list(slide.level_dimensions).index(slide.level_dimensions[-1]), (base_dims[0], base_dims[1])) #Always grab the second smallest layer


    layout = {"paper_bgcolor": "rgba(0, 0, 0, 0)",
              "plot_bgcolor": "rgba(0, 0, 0, 0)", }

    fig = go.Figure(layout=layout)
    #fig = go.FigureWidget(data=[{'x': [0, base_dims[0]], 
    #                                           'y': [0, base_dims[1]], 
    #                                           'mode': 'markers',
    #                                           'marker': {'opacity': 0}}], # invisible trace to init axes and to support autoresize
    #                                    layout={'width': plot_size, 'height': plot_size, 'yaxis' : dict(scaleanchor = "x", scaleratio = 1)})  
    fig.add_trace(
        go.Scatter(
            x=[0, base_dims[0] * scale_factor],
            y=[0, base_dims[1] * scale_factor],
            mode="markers",
            marker_opacity=0
        )
    )

    img_width = base_dims[0]
    img_height = base_dims[1]

    # Add image
    fig.add_layout_image(
        dict(
            x=0,
            sizex=img_width * scale_factor,
            y=img_height * scale_factor,
            sizey=img_height * scale_factor,
            xref="x",
            yref="y",
            opacity=1.0,
            layer="below",
            sizing="stretch",
            source=img_arr)
    )
    # Configure axes
    fig.update_xaxes(
        visible=False,
        range=[0, img_width * scale_factor],
        automargin=True
    )

    fig.update_yaxes(
        visible=False,
        range=[0, img_height * scale_factor],
        # the scaleanchor attribute ensures that the aspect ratio stays constant
        scaleanchor="x",
        automargin=True
    )

    # Get HTML representation of plotly.js and this figure
    plot_div = plot(fig, output_type='div', include_plotlyjs=True,config={'displaylogo': False,
                     'doubleClick':'reset',
                     'modeBarButtonsToAdd':['drawline',
                                            'drawopenpath',
                                            'drawclosedpath',
                                            'drawcircle',
                                            'drawrect',
                                            'eraseshape',
                                           ]})

    # Get id of html div element that looks like
    # <div id="301d22ab-bfba-4621-8f5d-dc4fd855bb33" ... >
    res = re.search('<div id="([^"]*)"', plot_div)
    div_id = res.groups()[0]

    # Build JavaScript callback for handling clicks

    # and opening the URL in the trace's customdata 
    js_callback = """
    <script>
    var plot_element = document.getElementById("{div_id}");
    plot_element.on('plotly_click', function(data){{
        console.log(data);
        var point = data.points[0];
        if (point) {{
            console.log(point.customdata);
            window.open(point.customdata);
        }}
    }})
    </script>
    """.format(div_id=div_id)

    # Build HTML string
    html_str = """
    <html>
    <body>
    {plot_div}
    {js_callback}
    </body>
    </html>
    """.format(plot_div=plot_div, js_callback=js_callback)


    # Write out HTML file
    out_name = str(outpath) +"/"+ scan_name + ".html"
    with open(out_name, 'w') as f:
        f.write(html_str)   

# Looped image extractor with ROI overlay

In [ ]:
from plotly.offline import plot
import pandas as pd
import plotly.graph_objs as go
import os
import openslide
import re
import numpy as np
from PIL import Image, ImageOps

angle = 22

#Append ROI identity as pos argument
pos = summary["SegmentDisplayName"].str.split("|").str[1]
summary["ROI_pos"] = pos

# Extract ROI info from the summary data
summary.head(3)
#Extract ROI, size info , filename conversion pathology and region info
summary_state = summary[summary["ScanName"].isin([scan_name])]
summary_state = summary_state[["SlideName","ScanName","ROILabel","AOISurfaceArea","ROICoordinateX","ROICoordinateY","pathology","region"]]
summary_state["ROI_add"] = "../DR_plots/categories/"+summary_state["SlideName"]+"/"+summary_state["ScanName"] + "-" + summary["ROI_pos"] +".png"

#Flip x and y
x = summary_state["ROICoordinateX"].copy()
summary_state["ROICoordinateY"] = summary_state["ROICoordinateY"]*-1
y = (summary_state["ROICoordinateY"]).copy()
summary_state["ROICoordinateX"] = y[:]
summary_state["ROICoordinateY"] = x[:]


#self = slide_tiler_proto_module()
plot_size = 1500
scale_factor = 1
plot_size = plot_size

scan_add = tif_locale + scan_name + ".tif"
slide = openslide.open_slide(scan_add)
base_dims = slide.level_dimensions[-1] #-2 for 2nd smallest
base_ds = slide.level_downsamples[-1]
img_arr = slide.read_region((0,0), list(slide.level_dimensions).index(slide.level_dimensions[-1]), (base_dims[0], base_dims[1])) #Always grab the second smallest layer
#Mirror
#img_arr= ImageOps.mirror(img_arr)
#flip
#img_arr= img_arr.rotate(angle)
#img_arr = img_arr.rotate(180)

#Automatically determine scaling factor for ROI points
X_scale = slide.level_dimensions[0][0]/slide.level_dimensions[-1][0]
Y_scale = slide.level_dimensions[0][1]/slide.level_dimensions[-1][1]
#summary_state["ROICoordinateX"]  = np.interp(summary_state["ROICoordinateX"] , (0, base_dims[0]), (0, base_dims[0]))
summary_state["ROICoordinateX"] = summary_state["ROICoordinateX"]/16#X_scale
summary_state["ROICoordinateY"] = summary_state["ROICoordinateY"]/15#Y_scale
summary_state["ROICoordinateY"] = summary_state["ROICoordinateY"] - 100
#summary_state["ROICoordinateX"] = summary_state["ROICoordinateX"] - 200

layout = {"paper_bgcolor": "rgba(0, 0, 0, 0)",
          "plot_bgcolor": "rgba(0, 0, 0, 0)", }
fig = go.Figure(layout=layout)
#fig = go.FigureWidget(data=[{'x': [0, base_dims[0]], 
#                                           'y': [0, base_dims[1]], 
#                                           'mode': 'markers',
#                                           'marker': {'opacity': 0}}], # invisible trace to init axes and to support autoresize
#                                    layout={'width': plot_size, 'height': plot_size, 'yaxis' : dict(scaleanchor = "x", scaleratio = 1)})  
fig.add_trace(
    go.Scatter(
        x=[0, base_dims[0] * scale_factor],
        y=[0, base_dims[1] * scale_factor],
        mode="markers",
        marker_opacity=0
    )
)

img_width = base_dims[0]
img_height = base_dims[1]

#Add the scatter trace for ROIs
size = summary_state["AOISurfaceArea"]/150#(X_scale*X_scale)
for i in (summary_state["region"].unique()):
    #Add scatter trace
    fig.add_trace(go.Scatter(
        name=i,
        x=summary_state["ROICoordinateX"][summary_state["region"].isin([i])],
        y=summary_state["ROICoordinateY"][summary_state["region"].isin([i])],
        mode='markers',marker_symbol='circle-open',#'circle-open'
        marker=dict(
        size=size,
        sizemode='area',
        reversescale = False,
        autocolorscale = True,
        #sizeref=2.*max(size)/(40.**2),
        #sizemin=4
        line=dict(
                color='MediumPurple',
                width=5
        )
        ),
        customdata=summary_state["ROI_add"],
)
)

# Add image
fig.add_layout_image(
    dict(
        x=0,
        sizex=img_width * scale_factor,
        y=img_height * scale_factor,
        sizey=img_height * scale_factor,
        xref="x",
        yref="y",
        opacity=1.0,
        layer="below",
        sizing="stretch",
        source=img_arr)
)
# Configure axes
fig.update_xaxes(
    visible=False,
    range=[0, img_width * scale_factor],
    automargin=True
)

fig.update_yaxes(
    visible=False,
    range=[0, img_height * scale_factor],
    # the scaleanchor attribute ensures that the aspect ratio stays constant
    scaleanchor="x",
    automargin=True
)



# Get HTML representation of plotly.js and this figure
plot_div = plot(fig, output_type='div', include_plotlyjs=True,config={'displaylogo': False,
                 'doubleClick':'reset',
                 'modeBarButtonsToAdd':['drawline',
                                        'drawopenpath',
                                        'drawclosedpath',
                                        'drawcircle',
                                        'drawrect',
                                        'eraseshape',
                                       ]})

# Get id of html div element that looks like
# <div id="301d22ab-bfba-4621-8f5d-dc4fd855bb33" ... >
res = re.search('<div id="([^"]*)"', plot_div)
div_id = res.groups()[0]

# Build JavaScript callback for handling clicks

# and opening the URL in the trace's customdata 
js_callback = """
<script>
var plot_element = document.getElementById("{div_id}");
plot_element.on('plotly_click', function(data){{
    console.log(data);
    var point = data.points[0];
    if (point) {{
        console.log(point.customdata);
        window.open(point.customdata);
    }}
}})
</script>
""".format(div_id=div_id)

# Build HTML string
html_str = """
<html>
<body>
{plot_div}
{js_callback}
</body>
</html>
""".format(plot_div=plot_div, js_callback=js_callback,js_buttons=js_buttons)

#Find and replace the diaply logo option 
#html_str = html_str.replace('displaylogo:!0', 'displaylogo:!1')
#html_str = re.sub('displaylogo:*', 'displaylogo:!1', html_str)


#fig.layout.images = [go.layout.Image(
#source = img_arr,  # plotly now performs auto conversion of PIL image to png data URI
#xref = "x",
#yref = "y",
#x = 0,
#y = base_dims[1],
#sizex = base_dims[0],
#sizey = base_dims[1],
#sizing = "stretch",
#layer = "below")]
#fig.update_layout(plot_bgcolor='rgba(0,0,0,0)',xaxis_showgrid=False, yaxis_showgrid=False, xaxis_zeroline=False, yaxis_zeroline=False);        
#fig.layout.on_change(update_image, 'xaxis.range', 'yaxis.range', 'width', 'height')


# Write out HTML file
with open(out_name, 'w') as f:
    f.write(html_str)